In [28]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from cvxopt import matrix 
from cvxopt import solvers

df_train = pd.read_csv("train.csv")
test_set  = pd.read_csv("test.csv")

# Splitting the training data into train and validation for the purposes of training and hyperparameter tuning 
training_set = df_train.iloc[:4000]
validation_set = df_train.iloc[4000:]

In [29]:
# Splitting data into class labels and replacing with <-1,1>
def split(data):
    y = data.iloc[:, 0].replace(0, -1).to_numpy().astype(float)
    X = data.iloc[:, 1:].to_numpy().astype(float)
    return y, X

In [30]:
# Implementations of Kernel functions
def linear_kernel(X, Z):
    return X @ Z.T

def polynomial_kernel(X, Z, C, D):
    return (X @ Z.T + C)**D 

def gaussian_kernel(X, Y, sigma):
    return np.exp((-linalg.norm(x-y)**2)/(2*(sigma**2)))

def rbf_kernel(X, Z, gamma):
    X2 = (X**2).sum(axis=1, keepdims=True)    
    Z2 = (Z**2).sum(axis=1, keepdims=True).T  
    d2 = X2 + Z2 - 2.0 * (X @ Z.T)
    return np.exp(-gamma * np.clip(d2, 0.0, None))

In [31]:
# Fitting data for improved prediction and seperation
y_train, X_train = split(training_set)
y_validation, X_validation = split(validation_set)
y_test, X_test = split(test_set)

scaler = StandardScaler()   

X_train = scaler.fit_transform(X_train)   
X_validation = scaler.transform(X_validation)
X_test  = scaler.transform(X_test)

In [32]:
# Fitting an SVM using the RBF Kernel
def fit_svm_rbf(X, y, C, gamma):
    n = X.shape[0]
    K = rbf_kernel(X, X, gamma)
    Q = np.outer(y,y) * K
    P = matrix(Q)
    q = matrix(-np.ones(n))
    G = matrix(np.vstack([-np.eye(n), np.eye(n)]))
    h = matrix(np.hstack([np.zeros(n), C*np.ones(n)]))
    A = matrix(y.astype(np.double), (1,n))
    b = matrix(0.0)

    solvers.options['show_progress'] = False
    sol = solvers.qp(P, q, G, h, A, b)
    alpha = np.ravel(sol['x'])

    eps = 1e-6
    sv = alpha > eps
    sv_free = (alpha > eps) & (alpha < C - eps)

    if np.any(sv_free):
        b_val = y[sv_free] - (alpha*y)[sv][:,None].T @ K[sv][:,sv_free]
        b_val = np.mean(b_val)
    else:
        b_val = y[sv] - (alpha*y)[sv][:,None].T @ K[sv][:,sv]
        b_val = np.mean(b_val)

    model = {'Xsv': X[sv], 'ysv': y[sv], 'alpha_sv': alpha[sv], 'b': float(b_val), 'gamma': gamma}
    return model

In [33]:
# Decision function returns the sum of a given X input 
def decision_function(model, Xquery):
    Kq = rbf_kernel(Xquery, model['Xsv'], model['gamma'])
    s = Kq @ (model['alpha_sv'] * model['ysv']) + model['b']
    return s

# Predict takes the sign of the decision sum
def predict(model, Xquery):
    return np.sign(decision_function(model, Xquery))

In [34]:
# Hyperparamater tuning to find optimal C and gamma values 
Cs = 10.0 ** np.arange(-2, 4)
gammas = 10.0 ** np.arange(-3, 2)

best = {'acc': -1, 'C': None, 'gamma': None, 'model': None}
for C in Cs:
    for gamma in gammas:
        mdl = fit_svm_rbf(X_train, y_train, C, gamma)
        yhat_val = predict(mdl, X_validation)
        acc = (yhat_val == y_validation).mean()
        if acc > best['acc']:
            best = {'acc': acc, 'C': C, 'gamma': gamma, 'model': mdl}

print("Optimal C value: ", best['C'])
print("Optimal RBF gamma: ", best['gamma'])
print("Validation accuracy: ", best['acc'])

Optimal C value:  10.0
Optimal RBF gamma:  0.001
Validation accuracy:  0.9746610357857302


Optimal C value:  10.0
Optimal RBF gamma:  0.001
Validation accuracy:  0.9746610357857302

In [35]:
# Retraining the SVM model fitted with the tuned C and gamma values and evaluating once against test
X_tv = np.vstack([X_train, X_validation])
y_tv = np.hstack([y_train, y_validation])

final_model = fit_svm_rbf(X_tv, y_tv, best['C'], best['gamma'])
test_pred = predict(final_model, X_test)
test_acc = (test_pred == y_test).mean()
print("Final SVM testing accuracy: ", test_acc)

Final SVM testing accuracy:  0.9773182121414277


Final SVM testing accuracy:  0.9773182121414277